In [23]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import os
os.environ["OMP_NUM_THREADS"] = "1"
import cv2
import numpy as np

In [24]:
# Загрузка изображений из директории
def load_images_from_directory(directory, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Масштабируем изображение к нужному размеру
                img_resized = cv2.resize(img, target_size)
                img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)  # Преобразуем в градации серого
                images.append(img_gray.flatten())  # Преобразуем изображение в одномерный массив
    images = np.array(images)
    return images

In [25]:
# Обучение K-Means на нормальных данных
def train_kmeans(normal_data, num_clusters=5):
    # Обучаем модель KMeans
    model = KMeans(n_clusters=num_clusters, n_init=10)
    model.fit(normal_data)

    return model


In [26]:
# Проверка изображения на аномалии
def is_anomaly(image, model):
    img_resized = cv2.resize(image, (128, 128))
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    img_flattened = img_gray.flatten().reshape(1, -1)  # Преобразуем изображение в одномерный массив

    closest_cluster, min_distances = pairwise_distances_argmin_min(img_flattened, model.cluster_centers_)
    # Аномалия, если расстояние до ближайшего кластера превышает порог
    return min_distances[0] > 1000  # Порог нужно выбрать экспериментально


In [28]:
# Основной код
if __name__ == "__main__":
    # Путь к данным
    data_dir = "object3_cut"

    # Загрузка нормальных изображений
    normal_images = load_images_from_directory(data_dir)

    # Обучение модели KMeans
    kmeans_model = train_kmeans(normal_images)

    # Проверка новых изображений на наличие аномалий
    source_dir = "object3_bad"
    for filename in os.listdir(source_dir):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
            img_path = os.path.join(source_dir, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue

            if is_anomaly(img, kmeans_model):
                print(f"Аномалия обнаружена на изображении: {img_path}")
            else:
                print(f"Все в порядке на изображении: {img_path}")

C:\Users\Daria\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.20.07.png
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.29.39.png
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.30.36.png
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.32.31.png
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.35.23.png
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.37.17.png
Аномалия обнаружена на изображении: object3_bad\21.08.2024_12.28.00.png
Аномалия обнаружена на изображении: object3_bad\21.08.2024_12.51.52.png
